# 🔍 NeuralSparseSearchTool - Sparse Vector Search

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#8E44AD', 'primaryTextColor':'#fff', 'primaryBorderColor':'#7D3C98', 'lineColor':'#F39C12', 'secondaryColor':'#3498DB', 'tertiaryColor':'#27AE60', 'fontSize':'16px'}}}%%
graph TB
    A[👤 Query Text] --> B[🤖 Flow Agent]
    B --> C{🔍 NeuralSparseSearchTool}
    C --> D[🧠 Sparse Encoding Model]
    D --> E[⚡ Sparse Vector<br/>term: score pairs]
    E --> F[📊 Index Search]
    F --> G[🎯 Matched Documents]
    G --> H[📋 Ranked Results]
    
    style A fill:#3498DB,stroke:#2980B9,color:#fff
    style C fill:#8E44AD,stroke:#7D3C98,color:#fff
    style D fill:#E74C3C,stroke:#C0392B,color:#fff
    style E fill:#F39C12,stroke:#D68910,color:#fff
    style H fill:#27AE60,stroke:#229954,color:#fff
```

## 📚 Learning Objectives

1. ✅ Understand **sparse vectors** vs **dense vectors**
2. ✅ Use **neural sparse encoding** models
3. ✅ Create **rank_features** indices
4. ✅ Perform **efficient semantic search** with sparse representations
5. ✅ Compare **sparse search** advantages

---

## 🎯 What is NeuralSparseSearchTool?

**NeuralSparseSearchTool** performs semantic search using **sparse vectors**:
- ⚡ **Sparse Encoding**: Only non-zero term scores (e.g., {"cat": 0.8, "pet": 0.6})
- 🎯 **Interpretable**: Can see which terms matter
- 🚀 **Efficient**: Smaller storage than dense vectors
- 📊 **rank_features**: Special OpenSearch field type

**Sparse vs Dense Vectors**:
- Dense: [0.12, -0.45, 0.89, ...] (384-1536 dimensions, all filled)
- Sparse: {"term1": 0.8, "term2": 0.3} (only non-zero values)

---

## Step 1: Import Libraries

In [1]:
import sys
import json

sys.path.append('..')
from agent_helpers import (
    get_os_client,
    create_flow_agent,
    execute_agent,
    cleanup_resources
)

print("✅ Libraries imported!")

✅ Libraries imported!


## Step 2: Initialize Client

In [2]:
client = get_os_client()
print("✅ Client ready")

✅ Client ready


## Step 3: Register Sparse Encoding Model

We'll use a **sparse encoding model** from HuggingFace (e.g., opensearch-project/opensearch-neural-sparse-encoding-v1).

In [3]:
# Register sparse encoding model
model_name = "amazon/neural-sparse/opensearch-neural-sparse-encoding-v1"
model_version = "1.0.1"

register_response = client.transport.perform_request(
    'POST',
    '/_plugins/_ml/models/_register',
    body={
        "name": model_name,
        "version": model_version,
        "model_format": "TORCH_SCRIPT"
    }
)

print("Registration response:", json.dumps(register_response, indent=2))

# Extract model_id from response (might be 'task_id' or 'model_id')
sparse_model_id = None
if 'task_id' in register_response:
    task_id = register_response['task_id']
    print(f"📋 Registration task: {task_id}")
    
    # Wait for registration to complete
    import time
    for i in range(60):  # Increased to 60 attempts (2 minutes)
        task_status = client.transport.perform_request(
            'GET', f'/_plugins/_ml/tasks/{task_id}'
        )
        print(f"⏳ Attempt {i+1}: Task state = {task_status.get('state', 'UNKNOWN')}")
        
        if task_status['state'] == 'COMPLETED':
            sparse_model_id = task_status['model_id']
            print(f"📦 Registered sparse model: {sparse_model_id}")
            break
        elif task_status['state'] == 'FAILED':
            print(f"❌ Registration failed: {task_status.get('error', 'Unknown error')}")
            raise Exception("Model registration failed")
        time.sleep(2)
    
    if sparse_model_id is None:
        raise Exception("Model registration timed out")
else:
    sparse_model_id = register_response['model_id']
    print(f"📦 Registered sparse model: {sparse_model_id}")

# Deploy model
deploy_response = client.transport.perform_request(
    'POST',
    f'/_plugins/_ml/models/{sparse_model_id}/_deploy'
)
print(f"🚀 Deploy initiated: {json.dumps(deploy_response, indent=2)}")

# Wait for deployment
for i in range(60):  # Increased timeout
    status = client.transport.perform_request(
        'GET', f'/_plugins/_ml/models/{sparse_model_id}'
    )
    current_state = status.get('model_state', 'UNKNOWN')
    print(f"⏳ Deployment attempt {i+1}: {current_state}")
    
    if current_state == 'DEPLOYED':
        print("✅ Sparse model deployed!")
        break
    elif current_state == 'DEPLOY_FAILED':
        print(f"❌ Deployment failed: {status}")
        raise Exception("Model deployment failed")
    time.sleep(5)

Registration response: {
  "task_id": "AFtriZsBLQ1mV2UNmSnY",
  "status": "CREATED"
}
📋 Registration task: AFtriZsBLQ1mV2UNmSnY
⏳ Attempt 1: Task state = CREATED
⏳ Attempt 2: Task state = CREATED
⏳ Attempt 3: Task state = CREATED
⏳ Attempt 4: Task state = CREATED
⏳ Attempt 5: Task state = CREATED
⏳ Attempt 6: Task state = CREATED
⏳ Attempt 7: Task state = CREATED
⏳ Attempt 8: Task state = CREATED
⏳ Attempt 9: Task state = CREATED
⏳ Attempt 10: Task state = CREATED
⏳ Attempt 11: Task state = CREATED
⏳ Attempt 12: Task state = CREATED
⏳ Attempt 13: Task state = CREATED
⏳ Attempt 14: Task state = CREATED
⏳ Attempt 15: Task state = CREATED
⏳ Attempt 16: Task state = CREATED
⏳ Attempt 17: Task state = CREATED
⏳ Attempt 18: Task state = CREATED
⏳ Attempt 19: Task state = CREATED
⏳ Attempt 20: Task state = CREATED
⏳ Attempt 21: Task state = CREATED
⏳ Attempt 22: Task state = CREATED
⏳ Attempt 23: Task state = CREATED
⏳ Attempt 24: Task state = CREATED
⏳ Attempt 25: Task state = CREATED
⏳ Atte

## Step 4: Create Ingest Pipeline with Sparse Encoding

In [4]:
pipeline_id = "sparse_encoding_pipeline"

client.ingest.put_pipeline(
    id=pipeline_id,
    body={
        "description": "Sparse encoding pipeline",
        "processors": [
            {
                "sparse_encoding": {
                    "model_id": sparse_model_id,
                    "field_map": {
                        "text": "sparse_embedding"
                    }
                }
            }
        ]
    }
)

print(f"✅ Pipeline created: {pipeline_id}")

✅ Pipeline created: sparse_encoding_pipeline


## Step 5: Create Index with rank_features Field

In [5]:
index_name = "articles_sparse"

if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)

client.indices.create(
    index=index_name,
    body={
        "settings": {
            "index.default_pipeline": pipeline_id
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "category": {"type": "keyword"},
                "sparse_embedding": {"type": "rank_features"}  # Sparse vector field
            }
        }
    }
)

print(f"✅ Created index with rank_features: {index_name}")

✅ Created index with rank_features: articles_sparse


## Step 6: Ingest Sample Documents

In [6]:
documents = [
    {"text": "Machine learning algorithms can predict customer behavior", "category": "AI"},
    {"text": "Deep neural networks excel at image recognition tasks", "category": "AI"},
    {"text": "Cloud computing provides scalable infrastructure for businesses", "category": "Cloud"},
    {"text": "Kubernetes orchestrates containerized applications efficiently", "category": "DevOps"},
    {"text": "Natural language processing enables chatbots to understand users", "category": "AI"},
]

for doc in documents:
    client.index(index=index_name, body=doc, refresh=True)

print(f"✅ Ingested {len(documents)} documents with sparse embeddings")

✅ Ingested 5 documents with sparse embeddings


## Step 7: Create Agent with NeuralSparseSearchTool

In [7]:
tools = [{
    "type": "NeuralSparseSearchTool",
    "parameters": {
        "model_id": sparse_model_id,
        "index": index_name,
        "embedding_field": "sparse_embedding",
        "source_field": ["text", "category"],
        "doc_size": 3,
        "input": "${parameters.question}"
    }
}]

agent_id = create_flow_agent(
    client, "Sparse_Search_Agent",
    "Performs semantic search using sparse vectors",
    tools
)
print(f"✅ Agent created: {agent_id}")

   Registering flow agent: Sparse_Search_Agent...
   ✓ Agent registered: DVttiZsBLQ1mV2UNbSlA
✅ Agent created: DVttiZsBLQ1mV2UNbSlA


## Step 8: Test Case 1 - AI Query

In [8]:
parameters = {"question": "artificial intelligence and predictions"}

print("🔍 Query: artificial intelligence and predictions")
print("="*60)
response = execute_agent(client, agent_id, parameters)
print("\n📊 Top Results:")
print(json.dumps(response, indent=2))

🔍 Query: artificial intelligence and predictions

📊 Top Results:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Machine learning algorithms can predict customer behavior\",\"category\":\"AI\"},\"_id\":\"CFttiZsBLQ1mV2UNOCnp\",\"_score\":12.799283}\n{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Natural language processing enables chatbots to understand users\",\"category\":\"AI\"},\"_id\":\"DFttiZsBLQ1mV2UNOSnH\",\"_score\":5.7580047}\n{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Deep neural networks excel at image recognition tasks\",\"category\":\"AI\"},\"_id\":\"CVttiZsBLQ1mV2UNOSkk\",\"_score\":5.6482444}\n"
        }
      ]
    }
  ]
}


## Step 9: Test Case 2 - Container Technology

In [9]:
parameters = {"question": "container orchestration systems"}

print("🔍 Query: container orchestration systems")
print("="*60)
response = execute_agent(client, agent_id, parameters)
print("\n📊 Results:")
print(json.dumps(response, indent=2))

🔍 Query: container orchestration systems

📊 Results:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Kubernetes orchestrates containerized applications efficiently\",\"category\":\"DevOps\"},\"_id\":\"C1ttiZsBLQ1mV2UNOSmS\",\"_score\":17.86498}\n{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Cloud computing provides scalable infrastructure for businesses\",\"category\":\"Cloud\"},\"_id\":\"ClttiZsBLQ1mV2UNOSla\",\"_score\":3.895081}\n{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Natural language processing enables chatbots to understand users\",\"category\":\"AI\"},\"_id\":\"DFttiZsBLQ1mV2UNOSnH\",\"_score\":3.2714818}\n"
        }
      ]
    }
  ]
}


## Step 10: Test Case 3 - Conversational AI

In [10]:
parameters = {"question": "chatbots and language understanding"}

print("🔍 Query: chatbots and language understanding")
print("="*60)
response = execute_agent(client, agent_id, parameters)
print("\n📊 Results:")
print(json.dumps(response, indent=2))

🔍 Query: chatbots and language understanding

📊 Results:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Natural language processing enables chatbots to understand users\",\"category\":\"AI\"},\"_id\":\"DFttiZsBLQ1mV2UNOSnH\",\"_score\":27.292557}\n{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Machine learning algorithms can predict customer behavior\",\"category\":\"AI\"},\"_id\":\"CFttiZsBLQ1mV2UNOCnp\",\"_score\":4.908008}\n{\"_index\":\"articles_sparse\",\"_source\":{\"text\":\"Deep neural networks excel at image recognition tasks\",\"category\":\"AI\"},\"_id\":\"CVttiZsBLQ1mV2UNOSkk\",\"_score\":3.5265918}\n"
        }
      ]
    }
  ]
}


## 🎓 Key Takeaways

### What We Learned:

1. **Sparse vs Dense Vectors**:
   ```python
   # Dense vector (384 dimensions)
   [0.12, -0.45, 0.89, 0.23, ...] # All 384 values stored
   
   # Sparse vector (rank_features)
   {"ai": 0.8, "machine": 0.6, "learning": 0.5} # Only non-zero terms
   ```

2. **Advantages of Sparse Search**:
   | Aspect | Dense Vectors | Sparse Vectors |
   |--------|--------------|----------------|
   | Storage | 384-1536 floats | ~10-50 terms |
   | Interpretability | ❌ Black box | ✅ See terms |
   | Performance | Good | Very Good |
   | Index Size | Larger | Smaller |

3. **Configuration**:
   ```python
   {
       "type": "NeuralSparseSearchTool",
       "parameters": {
           "model_id": sparse_model_id,
           "index": index_name,
           "embedding_field": "sparse_embedding",  # rank_features type
           "source_field": ["text", "category"],
           "doc_size": 3,
           "input": "${parameters.question}"
       }
   }
   ```

4. **Sparse Encoding Pipeline**:
   ```python
   {
       "processors": [
           {
               "sparse_encoding": {
                   "model_id": sparse_model_id,
                   "field_map": {
                       "text": "sparse_embedding"
                   }
               }
           }
       ]
   }
   ```

5. **rank_features Field**:
   ```python
   "mappings": {
       "properties": {
           "sparse_embedding": {"type": "rank_features"}
       }
   }
   ```

### Use Cases:

- 🔍 **Semantic Search**: Understanding query intent
- 📊 **Document Retrieval**: Finding relevant content
- 🎯 **Question Answering**: Matching questions to answers
- 🚀 **Efficient Search**: When storage/speed matter

### When to Use Sparse vs Dense:

**Use Sparse When**:
- ✅ Need interpretability (see which terms matched)
- ✅ Storage is limited
- ✅ Speed is critical
- ✅ Text-based search

**Use Dense When**:
- ✅ Cross-lingual search
- ✅ Image/audio embeddings
- ✅ Complex semantic relationships
- ✅ Maximum accuracy needed

---

## 🧹 Cleanup

In [ ]:
# # cleanup_resources(
# #     client=client,
# #     agent_ids=[agent_id],
# #     model_ids=[sparse_model_id]
# # )
# # client.indices.delete(index=index_name)
# # client.ingest.delete_pipeline(id=pipeline_id)
# # print("✅ Cleanup complete!")

## 🚀 Next Steps

- **VectorDBTool**: Compare with dense vector search
- **RAGTool**: Combine sparse search with LLM generation
- **Hybrid Search**: Mix sparse + dense + keyword search

📚 [Neural Sparse Search Docs](https://opensearch.org/docs/latest/ml-commons-plugin/agents-tools/tools/neural-sparse-search-tool/)